In [ ]:
import pandas as pd
df_raw = pd.read_parquet("data.parquet")
df_raw

In [ ]:
res = []

for x in df_raw.iterrows():
    values = x[1]
    for data_key, values_key in [('headline', 'headline_sentiment'), ('summary', 'summary_sentiment')]:
        data = values[data_key]
        d = values[values_key]
        label = max(d, key=lambda item: d[item])
        label = label.replace('postive', 'pos').replace('negative', 'neg').replace('neutral', 'neu')
        res.append({'data': data, 'label': label})
df = pd.DataFrame(res)
df

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

X_train, X_test, y_train, y_test = train_test_split(df['data'], df['label'], test_size=0.2, random_state=12)


vectorizer = TfidfVectorizer(min_df = 5,
                             max_df = 0.8,
                             sublinear_tf = True,
                             use_idf = True)

train_vectors = vectorizer.fit_transform(X_train)
test_vectors = vectorizer.transform(X_test)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn import svm

X_train, X_test, y_train, y_test = train_test_split(df['data'], df['label'], test_size=0.2, random_state=12)

vectorizer = TfidfVectorizer(min_df=5, max_df=0.8, sublinear_tf=True, use_idf=True)
train_vectors = vectorizer.fit_transform(X_train)
test_vectors = vectorizer.transform(X_test)

param_grid = {'C': [0.1, 1, 10, 100, 200, 500, 1000], 'gamma': [0.5, 0.2, 0.1, 0.01, 0.001], 'kernel': ['linear', 'poly', 'rb', 'sigmoid']}

grid_search = GridSearchCV(estimator=svm.SVC(class_weight='balanced'), param_grid=param_grid, scoring='accuracy', cv=5)

grid_search.fit(train_vectors, y_train)

print(grid_search.best_params_)


In [ ]:
from sklearn import svm

classifier_linear = svm.SVC(kernel='poly', gamma=0.2, C=100, class_weight='balanced')

classifier_linear.fit(train_vectors, y_train)

In [ ]:
from sklearn.metrics import classification_report
prediction_linear = clf_balanced_fitted.predict(test_vectors)


report = classification_report(y_test, prediction_linear, output_dict=True)
report

In [ ]:
import pickle
with open('clf_balanced_fitted_optimized.pickle', 'wb') as handle:
    pickle.dump(classifier_linear, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
import pickle
with open('../vectorizer.pkl', 'wb') as handle:
    pickle.dump(vectorizer, handle, protocol=pickle.HIGHEST_PROTOCOL)